In [1]:
print('hello')

hello


In [2]:
import os
from dotenv import load_dotenv

from groq import Groq
load_dotenv()
 

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [3]:
# Set the system prompt
system_prompt = {
    "role": "system",
    "content":
    "You are a helpful assistant. You reply with very short answers."
}

# Initialize the chat history
chat_history = [system_prompt]


# Get user input from the console
user_input = 'How are you'

  # Append the user input to the chat history
chat_history.append({"role": "user", "content": user_input})

response = client.chat.completions.create(model="llama3-70b-8192",
                                            messages=chat_history,
                                            max_tokens=100,
                                            temperature=1.2)
  # Append the response to the chat history
chat_history.append({
      "role": "assistant",
      "content": response.choices[0].message.content
  })
  # Print the response
print("Assistant:", response.choices[0].message.content)

Assistant: I'm good!


In [4]:
import mlflow
# Set the system prompt
system_prompt = {
    "role": "system",
    "content":
    "You are a helpful assistant. You reply with very short answers."
}

# Initialize the chat history
chat_history = [system_prompt]
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Create a new MLflow Experiment
mlflow.set_experiment("Places")






# Get user input from the console
user_input = 'Tell me about Delhi'
with mlflow.start_run():
    mlflow.log_param('user_input',user_input) 
    # Append the user input to the chat history
    chat_history.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(model="llama3-70b-8192",
                                                messages=chat_history,
                                                max_tokens=100,
                                                temperature=1.2)
    # Append the response to the chat history
    chat_history.append({
        "role": "assistant",
        "content": response.choices[0].message.content
    })
    mlflow.log_param('AI output',response.choices[0].message.content) 
    # Print the response
    print("Assistant:", response.choices[0].message.content)

2025/02/27 04:20:33 INFO mlflow.tracking.fluent: Experiment with name 'Places' does not exist. Creating a new experiment.


Assistant: Capital of India, rich history, cultural hub, famous for cuisine, monuments like Red Fort & Qutub Minar.
🏃 View run unequaled-koi-90 at: http://127.0.0.1:5000/#/experiments/303638892987493777/runs/4cd9588fe2ed478eaa06a566b4911a02
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/303638892987493777


In [ ]:
class SimpleAgent:  
    def __init__(self):  
        self.memory = []  # Conversation history  
        self.tools = {} 
        self.user_input = None   # Registered tools  

    def register_tool(self, tool_name, tool_function):  
        self.tools[tool_name] = tool_function  

    def respond(self, user_input) -> str:  
            self.user_input = user_input
            # Check if Groq AI tool is registered and use it
            if "groq_ai" in self.tools:
                groq_response = self.tools["groq_ai"](self.user_input)
                return groq_response
            return f"I'm a simple agent. You said: {self.user_input}. How can I help?"

In [6]:
def respond(self, user_input):  
    self.memory.append({"user": user_input})  

    if "weather" in user_input:  
        return "I can check the weather. Please say your city."  
    else:  
        return "Sorry, I didn't understand that." 

In [7]:
from typing import Dict, List  

class MemoryManager:  
    def __init__(self):  
        self.short_term: List[Dict] = []  # Chat history  
        self.long_term: Dict = {}         # User preferences  

    def update_short_term(self, role: str, content: str):  
        self.short_term.append({"role": role, "content": content})  

In [34]:
import requests as re
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry



def weather_tool() -> str:  
    """Fetches weather for a Lat Long.  Returns: str."""  
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": 28.27,
        "longitude": 76.14,
        "hourly": "temperature_2m"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}

    hourly_data["temperature_2m"] = hourly_temperature_2m

    hourly_dataframe = pd.DataFrame(data = hourly_data)
    return hourly_dataframe.head(1)

In [35]:
response = weather_tool()

print(response)

Coordinates 28.25°N 76.125°E
Elevation 269.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                       date  temperature_2m
0 2025-02-26 00:00:00+00:00         17.5455


In [36]:
agent = SimpleAgent()  
agent.register_tool("get_weather", weather_tool)  

In [37]:
def respond(self, user_input):  
    if "weather" in user_input:  
        weather = self.tools["get_weather"]
        return weather  

In [39]:
with mlflow.start_run():  
    mlflow.log_param("tool", "get_weather")  
    mlflow.log_metric("response_time_ms", 320)  
 

🏃 View run stately-fly-400 at: http://127.0.0.1:5000/#/experiments/303638892987493777/runs/5b7f4fd52c734ac49506a4958202a9ed
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/303638892987493777


In [42]:

def call_groq_ai(query: str) -> str:
    # Append the user input to the chat history
    # Set the system prompt
    system_prompt = {
        "role": "system",
        "content":
        "You are a helpful assistant. You reply with very short answers."
    }

    # Initialize the chat history
    chat_history = [system_prompt]


     

    # Append the user input to the chat history
    chat_history.append({"role": "user", "content": query})

    response = client.chat.completions.create(model="llama3-70b-8192",
                                                messages=chat_history,
                                                max_tokens=100,
                                                temperature=1.2)
    # Append the response to the chat history
    chat_history.append({
        "role": "assistant",
        "content": response.choices[0].message.content
    })
    # Print the response
    return ("Assistant:", response.choices[0].message.content)



In [ ]:
from typing import Callable, Any, Dict, Optional, List, get_type_hints
from dataclasses import dataclass
import inspect
import urllib.request
import json
import mlflow
from groq import Groq
import os
import logging
from datetime import datetime
from typing import _GenericAlias
import pytz  # For time zone tool
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

load_dotenv()

# Initialize MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("AI Agent Toolkit")

@dataclass
class Tool:
    """A callable tool with metadata for the agent"""
    name: str
    description: str
    func: Callable[..., str]
    parameters: Dict[str, Dict[str, str]]
    
    def __call__(self, *args, **kwargs) -> str:
        """Execute the tool with MLflow logging"""
        with mlflow.start_run(nested=True):
            try:
                # Log tool parameters
                mlflow.log_params(kwargs)
                start_time = datetime.now()
                
                # Execute tool
                result = self.func(*args, **kwargs)
                
                # Log metrics
                execution_time = (datetime.now() - start_time).total_seconds()
                mlflow.log_metric("execution_time", execution_time)
                mlflow.log_text(result, "tool_output.txt")
                
                return result
            except Exception as e:
                mlflow.log_param("error", str(e))
                return f"Tool error: {str(e)}"

def parse_docstring_params(docstring: str) -> Dict[str, str]:
    """Parse parameter descriptions from docstring"""
    params = {}
    if not docstring:
        return params
    
    current_param = None
    for line in docstring.split('\n'):
        line = line.strip()
        if line.startswith("Parameters:"):
            current_param = "parameters"
        elif line.startswith("- "):
            parts = line[2:].split(":", 1)
            if len(parts) == 2:
                current_param = parts[0].strip()
                params[current_param] = parts[1].strip()
        elif current_param and line:
            params[current_param] += " " + line.strip()
    
    return params

def get_type_description(type_hint: Any) -> str:
    """Convert type hints to human-readable format"""
    if isinstance(type_hint, _GenericAlias):
        args = ", ".join(map(get_type_description, type_hint.__args__))
        return f"{type_hint._name}[{args}]"
    return getattr(type_hint, "__name__", str(type_hint))

def tool(name: str = None, requires_auth: bool = False):
    """Decorator to create Tool instances from functions"""
    def decorator(func: Callable[..., str]) -> Tool:
        nonlocal name
        tool_name = name or func.__name__
        doc = inspect.getdoc(func) or ""
        
        # Parse parameters
        param_docs = parse_docstring_params(doc)
        type_hints = get_type_hints(func)
        sig = inspect.signature(func)
        
        parameters = {}
        for param in sig.parameters.values():
            if param.name == "self":
                continue
                
            parameters[param.name] = {
                "type": get_type_description(type_hints.get(param.name, Any)),
                "description": param_docs.get(param.name, "No description"),
                "required": param.default == inspect.Parameter.empty
            }
        
        # Add auth requirement to description
        if requires_auth:
            doc += "\n\nRequires authentication: Yes"
        
        return Tool(
            name=tool_name,
            description=doc.split("\n\n")[0],
            func=func,
            parameters=parameters
        )
    return decorator

class Agent:
    """Core AI agent with tool integration and planning capabilities"""
    
    def __init__(self):
        self.client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
        self.tools: Dict[str, Tool] = {}
        self.conversation_history = []
        
    def add_tool(self, tool: Tool) -> None:
        """Register a tool with the agent"""
        self.tools[tool.name] = tool
        logger.info(f"Registered tool: {tool.name}")
        
    def create_system_prompt(self) -> str:
        """Generate the system prompt with current tool definitions"""
        tools_schema = []
        for tool in self.tools.values():
            tool_schema = {
                "name": tool.name,
                "description": tool.description,
                "parameters": {
                    param: {
                        "type": info["type"],
                        "description": info["description"],
                        "required": info["required"]
                    }
                    for param, info in tool.parameters.items()
                }
            }
            tools_schema.append(tool_schema)
        
        return f"""You are an AI assistant with access to tools. Your responsibilities:
1. Analyze user queries to determine if tools are needed
2. Select appropriate tools and parameters
3. Return JSON with tool calls or direct response

Available tools: {json.dumps(tools_schema, indent=2)}

Response format:
{{
    "requires_tool": boolean,
    "response": string (optional),
    "tools": [
        {{
            "name": string,
            "parameters": object
        }}
    ]
}}"""

    def process_query(self, query: str) -> Dict:
        """Use LLM to determine required actions"""
        messages = [
            {"role": "system", "content": self.create_system_prompt()},
            {"role": "user", "content": query}
        ]
        
        response = self.client.chat.completions.create(
            model="mixtral-8x7b-32768",
            messages=messages,
            temperature=0.3,
            response_format={"type": "json_object"}
        )
        
        try:
            return json.loads(response.choices[0].message.content)
        except json.JSONDecodeError as e:
            logger.error(f"Failed to parse LLM response: {e}")
            return {"error": "Failed to process query"}

    def execute(self, query: str) -> str:
        """Full execution flow with MLflow tracking"""
        with mlflow.start_run():
            mlflow.log_param("query", query)
            start_time = datetime.now()
            
            # Store conversation history
            self.conversation_history.append({"role": "user", "content": query})
            
            # Get LLM plan
            plan = self.process_query(query)
            if "error" in plan:
                return "Failed to process request"
            
            mlflow.log_dict(plan, "plan.json")
            
            if not plan.get("requires_tool", False):
                mlflow.log_metric("direct_response", 1)
                return plan.get("response", "No response generated")
            
            # Execute tools
            results = []
            for tool_call in plan.get("tools", []):
                tool_name = tool_call["name"]
                if tool_name not in self.tools:
                    logger.warning(f"Tool {tool_name} not found")
                    continue
                    
                tool = self.tools[tool_name]
                try:
                    result = tool(**tool_call["parameters"])
                    results.append(result)
                except Exception as e:
                    results.append(f"Error with {tool_name}: {str(e)}")
            
            # Log metrics
            execution_time = (datetime.now() - start_time).total_seconds()
            mlflow.log_metrics({
                "total_time": execution_time,
                "tools_used": len(results)
            })
            
            return "\n".join(results)

# ======================
# Tool Implementations
# ======================

@tool("currency_converter", requires_auth=True)
def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert between currencies using real exchange rates
    
    Parameters:
        - amount: The amount to convert
        - from_currency: 3-letter currency code (e.g. USD)
        - to_currency: Target currency code
    """
    api_key = os.getenv("CURRENCY_API_KEY")
    url = f"https://api.exchangerate.host/convert?from={from_currency}&to={to_currency}&amount={amount}"
    
    try:
        with urllib.request.urlopen(url) as response:
            data = json.load(response)
            if data.get("success"):
                return f"{amount} {from_currency} = {data['result']:.2f} {to_currency}"
            return "Conversion failed"
    except Exception as e:
        return f"API error: {str(e)}"

@tool("time_converter")
def convert_time(time_str: str, from_tz: str, to_tz: str) -> str:
    """Convert time between time zones
    
    Parameters:
        - time_str: Time in HH:MM format
        - from_tz: Source timezone (e.g. Europe/Paris)
        - to_tz: Target timezone (e.g. America/New_York)
    """
    try:
        time = datetime.strptime(time_str, "%H:%M")
        from_zone = pytz.timezone(from_tz)
        to_zone = pytz.timezone(to_tz)
        
        localized = from_zone.localize(time)
        converted = localized.astimezone(to_zone)
        return converted.strftime("%H:%M %Z")
    except Exception as e:
        return f"Time conversion error: {str(e)}"

@tool("weather_lookup", requires_auth=True)
def get_weather(city: str, country: str) -> str:
    """Get current weather conditions
    
    Parameters:
        - city: City name
        - country: 2-letter country code
    """
    api_key = os.getenv("WEATHER_API_KEY")
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city},{country}"
    
    try:
        with urllib.request.urlopen(url) as response:
            data = json.load(response)
            current = data["current"]
            return f"{current['temp_c']}°C, {current['condition']['text']}"
    except Exception as e:
        return f"Weather API error: {str(e)}"

# ======================
# Usage Example
# ======================

if __name__ == "__main__":
    # Initialize agent
    agent = Agent()
    
    # Register tools
    agent.add_tool(convert_currency)
    agent.add_tool(convert_time)
    agent.add_tool(get_weather)
    
    # Example queries
    queries = [
        "Convert 100 USD to EUR",
        "What time is 14:30 Europe/Paris in America/New_York?",
        "What's the weather in Tokyo, JP?"
    ]
    
    for query in queries:
        print(f"\nQuery: {query}")
        response = agent.execute(query)
        print(f"Response: {response}")
    
    print("\nMLflow Experiment Tracking:")
    print(f"Track results at: {mlflow.get_tracking_uri()}")